<a href="https://colab.research.google.com/github/mburkey3/Intro_to_ML/blob/main/Homework6/Homework6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.optim as optim
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import transforms

In [ ]:
# download and normalize the input images
from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'
tensor_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))
tensor_cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

100%|██████████| 170498071/170498071 [00:02<00:00, 85242907.97it/s]


Extracting ../data-unversioned/p1ch7/cifar-10-python.tar.gz to ../data-unversioned/p1ch7/
Files already downloaded and verified


In [ ]:
# Building the actual model
n_out = 10

model = nn.Sequential(
    nn.Linear(3072, 512),
    nn.Tanh(),
    nn.Linear(512, n_out),
    nn.LogSoftmax(dim=1)
)

In [ ]:
def training_classifer(epoch, opt, M, lossFn, trainData, div) :
    for E in range(1, epoch + 1) :
        for img, label in trainData :
            trainOut = M(img.view(img.shape[0], -1))
            trainLoss = lossFn(trainOut, label)

            opt.zero_grad()
            trainLoss.backward()
            opt.step()

        if E % div == 0:
            print(f"Epoch {E}, Training loss {trainLoss.item():.4f}")

    return

In [ ]:
def validation_test(M, valData) :
    correct = 0
    total = 0

    with torch.no_grad() :
        for img, label in valData :
            batchSize = img.shape[0]
            out = M(img.view(batchSize, -1))
            _, predict = torch.max(out, dim=1)
            total += label.shape[0]
            correct += int((predict == label).sum())

    print("Accuracy: ", correct / total)
    return

In [ ]:
# Training the model
optimizer = optim.SGD(model.parameters(), lr = 0.001)

trainLoader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64, shuffle=True) # This should speed up the process with mini batches

training_classifer(
    epoch = 10,
    opt = optimizer,
    M = model,
    lossFn = nn.NLLLoss(),
    trainData = trainLoader,
    div = 1
)

Epoch 1, Training loss 2.1732
Epoch 2, Training loss 1.8272
Epoch 3, Training loss 1.9203
Epoch 4, Training loss 2.0380
Epoch 5, Training loss 1.5556
Epoch 6, Training loss 1.8492
Epoch 7, Training loss 1.6213
Epoch 8, Training loss 1.6566
Epoch 9, Training loss 1.8129
Epoch 10, Training loss 1.6937


In [ ]:
# Validating the model
valLoader = torch.utils.data.DataLoader(tensor_cifar10_val, batch_size=64, shuffle=True) # This will seperate into mini batches to speed up the process

validation_test(
    M = model,
    valData = valLoader
)

Accuracy:  0.4135


In [ ]:
## PART B
# Rebuild model with extra layers
modelB = nn.Sequential(
    nn.Linear(3072, 1536),
    nn.Tanh(),
    nn.Linear(1536, 768),
    nn.Tanh(),
    nn.Linear(768, 384),
    nn.Tanh(),
    nn.Linear(384, n_out),
    nn.LogSoftmax(dim=1)
)

In [ ]:
# Train the model
optimizer = optim.SGD(modelB.parameters(), lr = 0.001)

training_classifer(
    epoch = 300,
    opt = optimizer,
    M = modelB,
    lossFn = nn.NLLLoss(),
    trainData = trainLoader,
    div = 50
)

Epoch 50, Training loss 1.7690
Epoch 100, Training loss 1.1511
Epoch 150, Training loss 0.4502
Epoch 200, Training loss 0.1442
Epoch 250, Training loss 0.0329
Epoch 300, Training loss 0.0090


In [ ]:
# Validating the larger model
validation_test(
    M = modelB,
    valData = valLoader
)

Accuracy:  0.4632
